In [1]:
from datasets import load_dataset

ds = load_dataset('billingsmoore/tibetan-asr', split='train').take(5000)

ds

Dataset({
    features: ['audio', 'transcription', 'speaker_id'],
    num_rows: 5000
})

In [2]:
ds.save_to_disk('nict-tib-1')

Saving the dataset (0/3 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [1]:
from datasets import load_from_disk

ds = load_from_disk('nict-tib-1')
ds

Dataset({
    features: ['audio', 'transcription', 'speaker_id'],
    num_rows: 5000
})

In [2]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, AutoTokenizer, Wav2Vec2Config
import torch

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("openpecha/Garchen_Rinpoche_stt")
tokenizer = AutoTokenizer.from_pretrained("openpecha/Garchen_Rinpoche_stt")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [3]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [5]:
ds = ds.select(range(10, 20)).map(prepare_dataset, num_proc=None, batched=False)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

: 

In [ ]:
ds.save_to_disk('../Data/nict-tib1')